In [2]:
!pip uninstall -y llama-index pydantic
!pip install "pydantic<2.0"
!pip install llama-index==0.10.33 faiss-cpu openai PyPDF2 python-docx

Found existing installation: pydantic 2.11.2
Uninstalling pydantic-2.11.2:
  Successfully uninstalled pydantic-2.11.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.3.51 requires pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.10.21 which is incompatible.
albumentations 2.0.5 requires pydantic>=2.9.2, but you have pydantic 1.10.21 which is incompatible.
google-genai 1.9.0 requires pydantic<3.0.0,>=2.0.0, but you have pydantic 1.10.21 which is incompatible.
thinc 8.3.6 requires pydantic<3.0.0,>=2.0.0, but you have pydantic 1.10.21 which is incompatible.
langchain 0.3.23 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.21 which is incompatible.
   

In [3]:
%pip install llama-index-vector-stores-faiss

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.6/443.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.21
    Uninstalling pydantic-1.10.21:
      Successfully uninstalled pydantic-1.10.21
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.10.68.post1
    Uninstalling llama-index-core-0.10.68.post1:
      Successfully uninstalled llama-index-core-0.10.68.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.10.33 requires llama-index-core<0.11.0,>=0.10.32, but you have llama-index-core 0.12.30 which is incompatible.

In [4]:
!pip install llama-index

  Using cached llama_index_core-0.10.68.post1-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.10.68.post1-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.12.30
    Uninstalling llama-index-core-0.12.30:
      Successfully uninstalled llama-index-core-0.12.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-vector-stores-faiss 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.10.68.post1 which is incompatible.


In [2]:
import os
import faiss
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.settings import Settings

from google.colab import files
uploaded = files.upload()  # Upload .pdf, .txt, or .docx

# 🔐 Set your OpenAI key
os.environ["OPENAI_API_KEY"] = ""
# ✅ Initialize LLM & Embedding
llm = OpenAI(model="gpt-3.5-turbo")
embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.llm = llm
Settings.embed_model = embed_model

# ✅ Create uploads folder and move files (if needed)
upload_dir = "/content"
os.makedirs(upload_dir, exist_ok=True)

# Move uploaded files to /content/uploads
for fname in uploaded.keys():
    os.rename(f"/content/{fname}", f"{upload_dir}/{fname}")

# ✅ Load & validate documents
documents = SimpleDirectoryReader(input_dir=upload_dir).load_data()
if not documents or all(len(doc.text.strip()) == 0 for doc in documents):
    raise ValueError("❗ No extractable text found. Try using .txt or OCR-processed PDFs.")

# ✅ Create FAISS index
dim = len(embed_model.get_text_embedding("test"))
faiss_index = faiss.IndexFlatL2(dim)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# ✅ Build RAG index
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
query_engine = index.as_query_engine(similarity_top_k=3)
print("✅ RAG index ready.")

Saving DE_Chandini 4.pdf to DE_Chandini 4 (1).pdf
✅ RAG index ready.


In [4]:
query = "Summarize the key qualifications in the resume."  # or any relevant question
response = query_engine.query(query)

print("🔍 Answer:\n", response)


🔍 Answer:
 The individual holds a Bachelor of Technology in Computer Science Engineering and is currently pursuing a Master of Science in Computer Software Engineering in Data Science. They have technical skills in various programming languages, operating systems, cloud services, databases, and tools. With experience as a Senior Data Engineer and Data Engineer, they have expertise in architecting ETL pipelines, SQL queries, data migration, and data quality checks. The individual has worked on projects involving recommendation systems, HR portals, advanced data models, and sales forecasting systems. Additionally, they have received certifications in cloud data quality, cloud data integration, and AWS Cloud Practitioner.
